In [0]:
from pyspark.sql.functions import col,rank
from pyspark.sql import Window
from pyspark.sql.types import IntegerType, BooleanType, DoubleType, DataType

In [0]:
#Key-vault secrets
secret_application_id = dbutils.secrets.get(scope="key-vault-scope",key="tokyo-olympic-application-id")
secret_directory_id = dbutils.secrets.get(scope="key-vault-scope",key="tokyo-olympic-directory-id")
secret_secret_key = dbutils.secrets.get(scope="key-vault-scope",key="tokyo-olympics-secret-key")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": f"{secret_application_id}",
"fs.azure.account.oauth2.client.secret": f'{secret_secret_key}',
"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{secret_directory_id}/oauth2/token"}


# Mounting Azure Data Lake storage into this data factory notebook

dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@tokyoolympicdatamonil.dfs.core.windows.net", # container@storageaccount
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

Out[39]: True

In [0]:
%fs
ls "mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1723355549000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1723355578000


In [0]:
%fs
ls "mnt/tokyoolympic/raw-data"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/athletes.csv,athletes.csv,418492,1723671034000
dbfs:/mnt/tokyoolympic/raw-data/coaches.csv,coaches.csv,16889,1723671050000
dbfs:/mnt/tokyoolympic/raw-data/entriesgender.csv,entriesgender.csv,946,1723671063000
dbfs:/mnt/tokyoolympic/raw-data/medals.csv,medals.csv,1912,1723671075000
dbfs:/mnt/tokyoolympic/raw-data/teams.csv,teams.csv,35270,1723671088000


In [0]:
## creation of spark session is not required as we are on azure databricks and it is done automatically
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
# athletes = (
#                 spark
#                 .read
#                 .format("csv")
#                 .option("header","True")
#                 .load("/mnt/tokyoolympic/raw-data/athletes.csv")
#             )

In [0]:
def get_csv(fname):
    return (
                spark
                .read
                .format("csv")
                .option("header","True")
                .option("inferSchema","True")
                .load(f"/mnt/tokyoolympic/raw-data/{fname}.csv")
            )

In [0]:
athletes = get_csv("athletes")
athletes.show()
athletes.printSchema()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
coaches = get_csv("coaches")
coaches.show()
coaches.printSchema()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [0]:
entriesgender = get_csv("entriesgender")
entriesgender.show()
entriesgender.printSchema()

+--------------------+------+----+
|          Discipline|Female|Male|
+--------------------+------+----+
|      3x3 Basketball|    32|  32|
|             Archery|    64|  64|
| Artistic Gymnastics|    98|  98|
|   Artistic Swimming|   105|   0|
|           Athletics|   969|1072|
|           Badminton|    86|  87|
|   Baseball/Softball|    90| 144|
|          Basketball|   144| 144|
|    Beach Volleyball|    48|  48|
|              Boxing|   102| 187|
|        Canoe Slalom|    41|  41|
|        Canoe Sprint|   123| 126|
|Cycling BMX Frees...|    10|   9|
|  Cycling BMX Racing|    24|  24|
|Cycling Mountain ...|    38|  38|
|        Cycling Road|    70| 131|
|       Cycling Track|    90|  99|
|              Diving|    72|  71|
|          Equestrian|    73| 125|
|             Fencing|   107| 108|
+--------------------+------+----+
only showing top 20 rows

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)



In [0]:
# entriesgender = (
#                     entriesgender
#                     .withColumn("Female",col("Female").cast(IntegerType()))
#                     .withColumn("Male",col("Male").cast(IntegerType()))
#                 )
# entriesgender.printSchema()

In [0]:
entriesgender = entriesgender.withColumn("Total", entriesgender.Female + entriesgender.Male)
entriesgender.show()
entriesgender.printSchema()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
medals = get_csv("medals")
medals.show()
medals.printSchema()

+----+--------------------+----+------+------+
|Rank|         TeamCountry|Gold|Silver|Bronze|
+----+--------------------+----+------+------+
|   1|United States of ...|  39|    41|    33|
|   2|People's Republic...|  38|    32|    18|
|   3|               Japan|  27|    14|    17|
|   4|       Great Britain|  22|    21|    22|
|   5|                 ROC|  20|    28|    23|
|   6|           Australia|  17|     7|    22|
|   7|         Netherlands|  10|    12|    14|
|   8|              France|  10|    12|    11|
|   9|             Germany|  10|    11|    16|
|  10|               Italy|  10|    10|    20|
|  11|              Canada|   7|     6|    11|
|  12|              Brazil|   7|     6|     8|
|  13|         New Zealand|   7|     6|     7|
|  14|                Cuba|   7|     3|     5|
|  15|             Hungary|   6|     7|     7|
|  16|   Republic of Korea|   6|     4|    10|
|  17|              Poland|   4|     5|     5|
|  18|      Czech Republic|   4|     4|     3|
|  19|       

In [0]:
# medals = (
#                     medals
#                     .withColumn("Gold",col("Gold").cast(IntegerType()))
#                     .withColumn("Silver",col("Silver").cast(IntegerType()))
#                     .withColumn("Bronze",col("Bronze").cast(IntegerType()))
#                 )
# medals.printSchema()

In [0]:
medals = medals.withColumn("Total", medals.Gold + medals.Silver + medals.Bronze)
w = Window.orderBy(medals["Total"].desc())
medals = medals.withColumn("Rank by Total",rank().over(w))
medals.show()
medals.printSchema()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   3|               Japan|  27|    14|    17|   58|            5|
|   6|           Australia|  17|     7|    22|   46|            6|
|  10|               Italy|  10|    10|    20|   40|            7|
|   9|             Germany|  10|    11|    16|   37|            8|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
teams = get_csv("teams")
teams.show()
teams.printSchema()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
# top countries with the highest number of Bronze medals
top_bronze_medal_countries = medals.orderBy("Bronze", ascending=False).select("TeamCountry","Bronze").show()

+--------------------+------+
|         TeamCountry|Bronze|
+--------------------+------+
|United States of ...|    33|
|                 ROC|    23|
|       Great Britain|    22|
|           Australia|    22|
|               Italy|    20|
|People's Republic...|    18|
|               Japan|    17|
|             Germany|    16|
|         Netherlands|    14|
|             Ukraine|    12|
|              France|    11|
|              Canada|    11|
|   Republic of Korea|    10|
|              Turkey|     9|
|              Brazil|     8|
|          Kazakhstan|     8|
|         New Zealand|     7|
|             Hungary|     7|
|               Spain|     6|
|         Switzerland|     6|
+--------------------+------+
only showing top 20 rows



In [0]:
# Percentage of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] /  entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] /  entriesgender['Total']
).show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
# athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed-data/athletes")

In [0]:
def sink_csv(df,name):
    df.repartition(1).write.mode("overwrite").option("header",'true').csv(f"/mnt/tokyoolympic/transformed-data/{name}")


sink_csv(athletes,"athletes")
sink_csv(coaches,"coaches")
sink_csv(entriesgender,"entriesgender")
sink_csv(medals,"medals")
sink_csv(teams,"teams")